In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import config
import matplotlib.dates as dates
from hydrophone_data_processing import load, useful_variables, plotting, signal_processing
from Event import Event


In [2]:

def import_catalog(file):
    df = pd.read_csv(file)
    df.sort_values(by='first_arrival', inplace=True)
    df['arrival_datetime'] = df.arrival_time.apply(dates.num2date)
    return df
    
# also need to remove a number of columns that are simply irrelevant to the analysis or untested for a shareable dataset.

df141 = import_catalog(file='141precision.csv')
df188 = import_catalog(file='188precision.csv')
df197 = import_catalog(file='197precision.csv')
df211 = import_catalog(file='211precision.csv')


columns_to_keep = ['id', 'depth', 'relative_depth',
        'first_hydrophone', 'second_hydrophone',
       'arrival_time', 'first_arrival', 'second_arrival', 'dt',
       'max_amp', 'cum_amp', 'arrival_datetime', 'origin_time']

df141 = df141[columns_to_keep].copy()
df188 = df188[columns_to_keep].copy()
df197 = df197[columns_to_keep].copy()
df211 = df211[columns_to_keep].copy()

df141.sort_values(by='arrival_datetime', inplace=True)
df188.sort_values(by='arrival_datetime', inplace=True)
df197.sort_values(by='arrival_datetime', inplace=True)
df211.sort_values(by='arrival_datetime', inplace=True)

# # removing the events with maximum amplitude <0.1 since they are false positives
df141 = df141[df141.max_amp > 0.5].copy()
df188 = df188[df188.max_amp > 0.5].copy()
df197 = df197[df197.max_amp > 0.5].copy()
df211 = df211[df211.max_amp > 0.5].copy()

df141.drop_duplicates(subset=['arrival_datetime',], inplace=True)
df188.drop_duplicates(subset=['arrival_datetime',], inplace=True)
df197.drop_duplicates(subset=['arrival_datetime',], inplace=True)
df211.drop_duplicates(subset=['arrival_datetime',], inplace=True)

df141 = df141[df141.depth > 160].copy()
df188 = df188[df188.depth > 160].copy()
df197 = df197[df197.depth > 160].copy()
df211 = df211[df211.depth > 160].copy()

In [3]:
df211[df211.max_amp>100]

,id,depth,relative_depth,first_hydrophone,second_hydrophone,arrival_time,first_arrival,second_arrival,dt,max_amp,cum_amp,arrival_datetime,origin_time
28,28,171.750,1.750,2,3,18107.946550,2019-07-30 22:43:01.920999+00:00,2019-07-30 22:43:01.958999+00:00,-0.038,239.604612,552.399078,2019-07-30 22:43:01.920999+00:00,2019-07-30T22:43:01.919999Z
32,32,171.750,1.750,2,3,18107.946887,2019-07-30 22:43:31.079999+00:00,2019-07-30 22:43:31.117999+00:00,-0.038,148.537548,741.936307,2019-07-30 22:43:31.079999+00:00,2019-07-30T22:43:31.078999Z
45,45,178.750,8.750,2,3,18107.947428,2019-07-30 22:44:17.770999+00:00,2019-07-30 22:44:17.800999+00:00,-0.030,265.773237,499.599131,2019-07-30 22:44:17.770999+00:00,2019-07-30T22:44:17.765999Z
51,51,182.250,12.250,2,3,18107.947632,2019-07-30 22:44:35.408999+00:00,2019-07-30 22:44:35.434999+00:00,-0.026,557.253460,1195.600673,2019-07-30 22:44:35.408999+00:00,2019-07-30T22:44:35.401999Z
52,52,182.250,12.250,2,3,18107.947637,2019-07-30 22:44:35.855999+00:00,2019-07-30 22:44:35.881999+00:00,-0.026,140.246880,308.833661,2019-07-30 22:44:35.855999+00:00,2019-07-30T22:44:35.848999Z
70,70,189.250,19.250,2,3,18107.948034,2019-07-30 22:45:10.146999+00:00,2019-07-30 22:45:10.164999+00:00,-0.018,163.083259,379.767390,2019-07-30 22:45:10.146999+00:00,2019-07-30T22:45:10.135999Z
107,107,247.875,77.875,3,2,18107.949635,2019-07-30 22:47:28.428999+00:00,2019-07-30 22:47:28.477999+00:00,-0.049,131.840412,419.692079,2019-07-30 22:47:28.428999+00:00,2019-07-30T22:47:28.384499Z
111,111,218.125,48.125,3,2,18107.949662,2019-07-30 22:47:30.756999+00:00,2019-07-30 22:47:30.771999+00:00,-0.015,362.396986,1216.227268,2019-07-30 22:47:30.756999+00:00,2019-07-30T22:47:30.729499Z
119,119,219.875,49.875,3,2,18107.949768,2019-07-30 22:47:39.966999+00:00,2019-07-30 22:47:39.983999+00:00,-0.017,156.502067,379.467305,2019-07-30 22:47:39.966999+00:00,2019-07-30T22:47:39.938499Z
134,134,238.250,68.250,3,2,18107.950575,2019-07-30 22:48:49.690999+00:00,2019-07-30 22:48:49.728999+00:00,-0.038,445.612917,1012.650063,2019-07-30 22:48:49.690999+00:00,2019-07-30T22:48:49.651999Z


In [4]:
paths = useful_variables.make_hydrophone_data_paths(borehole='a', year=2019, julian_day=211)

# loads data for all hydrophones
# converts to pascals
# flips the sign on hydrophone 3 if there it is borehole B due to wiring problem
waveforms = load.import_corrected_data_for_single_day(paths=paths)
print('loading data from:', paths)

loading data from: ['/media/sda/data/robdata/Hydrophones/DAYS/A00/A00.7F.01.GDH.2019.211', '/media/sda/data/robdata/Hydrophones/DAYS/A00/A00.7F.02.GDH.2019.211', '/media/sda/data/robdata/Hydrophones/DAYS/A00/A00.7F.03.GDH.2019.211', '/media/sda/data/robdata/Hydrophones/DAYS/A00/A00.7F.04.GDH.2019.211', '/media/sda/data/robdata/Hydrophones/DAYS/A00/A00.7F.05.GDH.2019.211', '/media/sda/data/robdata/Hydrophones/DAYS/A00/A00.7F.06.GDH.2019.211']


In [5]:
waveforms

6 Trace(s) in Stream:
7F.A00.01.GDH | 2019-07-30T00:00:05.142999Z - 2019-07-31T00:00:04.430999Z | 1000.0 Hz, 86399289 samples
7F.A00.02.GDH | 2019-07-30T00:00:02.589999Z - 2019-07-31T00:00:01.721999Z | 1000.0 Hz, 86399133 samples
7F.A00.03.GDH | 2019-07-30T00:00:02.712999Z - 2019-07-31T00:00:03.635999Z | 1000.0 Hz, 86400924 samples
7F.A00.04.GDH | 2019-07-30T00:00:03.202999Z - 2019-07-31T00:00:01.390999Z | 1000.0 Hz, 86398189 samples
7F.A00.05.GDH | 2019-07-30T00:00:01.154999Z - 2019-07-31T00:00:03.780999Z | 1000.0 Hz, 86402627 samples
7F.A00.06.GDH | 2019-07-30T00:00:01.511999Z - 2019-07-31T00:00:00.377999Z | 1000.0 Hz, 86398867 samples

In [6]:
df211[df211.id==300]

,id,depth,relative_depth,first_hydrophone,second_hydrophone,arrival_time,first_arrival,second_arrival,dt,max_amp,cum_amp,arrival_datetime,origin_time
300,300,378.25,68.25,5,4,18107.958229,2019-07-30 22:59:50.983999+00:00,2019-07-30 22:59:51.021999+00:00,-0.038,170.523931,1199.04407,2019-07-30 22:59:50.983999+00:00,2019-07-30T22:59:50.944999Z


In [20]:
df211.loc[300]

id                                                300
depth                                          378.25
relative_depth                                  68.25
first_hydrophone                                    5
second_hydrophone                                   4
arrival_time                             18107.958229
first_arrival        2019-07-30 22:59:50.983999+00:00
second_arrival       2019-07-30 22:59:51.021999+00:00
dt                                             -0.038
max_amp                                    170.523931
cum_amp                                    1199.04407
arrival_datetime     2019-07-30 22:59:50.983999+00:00
origin_time               2019-07-30T22:59:50.944999Z
Name: 300, dtype: object

In [29]:
# e = df211.loc[300]
# waveforms.plot(
#     starttime=obspy.UTCDateTime(pd.to_datetime(e.first_arrival))-0.2
#     , endtime=obspy.UTCDateTime(pd.to_datetime(e.second_arrival))+0.5
#                                )

# waveforms[5].data.max()

def 

13.734678370219322

In [18]:
import obspy
id = 300
# df_picks_row = df211.loc[id]
df_picks_row = df211[df211.id==id].loc[id]
e = Event(id=id
          , starttime=obspy.UTCDateTime(pd.to_datetime(df_picks_row.first_arrival))
          , init_first_hphone=df_picks_row.first_hydrophone
          , waveforms=waveforms.copy()
          , velocity_model=1750
         )
e.data

TypeError: unsupported operand type(s) for *: 'UTCDateTime' and 'float'

In [16]:
df_picks_row.first_arrival

'2019-07-30 22:59:50.983999+00:00'

In [14]:
df211[df211.id==id].loc[300]

id                                                300
depth                                          378.25
relative_depth                                  68.25
first_hydrophone                                    5
second_hydrophone                                   4
arrival_time                             18107.958229
first_arrival        2019-07-30 22:59:50.983999+00:00
second_arrival       2019-07-30 22:59:51.021999+00:00
dt                                             -0.038
max_amp                                    170.523931
cum_amp                                    1199.04407
arrival_datetime     2019-07-30 22:59:50.983999+00:00
origin_time               2019-07-30T22:59:50.944999Z
Name: 300, dtype: object